In [1]:
import argparse
import pandas as pd
import numpy as np
import peddy ### must be in python version 3.7 for peddy to actually work


In [ ]:
ped = peddy.Ped('lynchWGS.ped.txt', 'Paternal_ID' == str) ###import peddy file

In [ ]:
def has_parents(sample):
    """Check if Peddy sample has both parents in ped file"""
    if sample.mom is not None and sample.dad is not None:
        return True
    return False

In [ ]:
df = pd.read_table('LaurelSTRsupdated.tsv', delim_whitespace = True, index_col = 0, dtype = {'sample' : str})

In [ ]:
dfnew = df[['allele1_est', 'allele2_est']]

In [ ]:
dfnewallele1sorted = dfnew.sort_values(by=['allele1_est'])

In [ ]:
newdf2 = dfnew.groupby(['allele2_est', 'allele1_est']).first()

In [ ]:
newdf2

In [ ]:
dad_alleles = [0.00, 152]

In [ ]:
mom_alleles = [300, 0.00]

In [ ]:
parent_alleles = ['NaN', 'NaN', 'NaN', 'NaN']

In [ ]:
kid_alleles = [3.67, 5000]

In [ ]:
kid_alleles = [0.00, 4000]

In [ ]:
def valid_alleles(p_alleles):
    if len(p_alleles) < 2:
        yield []
        return
    if len(p_alleles) % 2 == 1:
        # Handle odd length list
        for i in range(len(p_alleles)):
            for result in valid_alleles(p_alleles[:i] + p_alleles[i+1:]):
                yield result
    else:
        a = p_alleles[0]
        for i in range(1,len(p_alleles)):
            pair = (a,p_alleles[i])
            for rest in valid_alleles(p_alleles[1:i]+p_alleles[i+1:]):
                yield [pair] + rest

In [ ]:
allele_list = []
for x in valid_alleles(mom_alleles):
        allele_list.append(x)

In [ ]:
allele_list

In [ ]:
kiddict = {
    "allele1": 400,
    'allele2': 0,
}

In [ ]:
momdict = {
    "allele1": 400,
    'allele2': 0,
}

In [ ]:
momdict["allele1"] 

In [ ]:
kiddict["allele1"]

In [ ]:
fruits = ["apple", "banana", "cherry", "kiwi", "mango"]
newlist = []

for x in mom_alleles:
  if "a" in x:
    newlist.append(x)

print(newlist)


In [ ]:
#def compare_alleles(parents, kids):
 #   allele_list = []
  #  for x in valid_alleles(parent_alleles):
   #     allele_list.append(x)
    #flat_list = list(np.concatenate(allele_list).flat)
    #if len(set(flat_list).intersection(kid_alleles)) > 0:
     #   print('Possible match!')
      #  print(set(flat_list).intersection(kid_alleles))
    #else:
     #   print('No match')
    ### this can't work because sets eliminate the duplicates

In [ ]:
def compare_alleles(mom, dad, kid):
    results = {}
    for i in mom_alleles:
        results[i] = kid_alleles.count(i) 
    print(results)
    
    results = {}
    for i in dad_alleles:
        results[i] = kid_alleles.count(i) 
    print(results)
    

In [ ]:
allele_list = []
for x in valid_alleles(parent_alleles):
    allele_list.append(x)

In [ ]:
flat_list = list(np.concatenate(allele_list).flat)

In [ ]:
sorted(flat_list)

In [ ]:
compare_alleles(mom_alleles, dad_alleles, kid_alleles)
##could do loop
## could do list comprehension but


    

In [ ]:
sum = 0
sum += dict[key]
## for key in dictionary:

In [ ]:
momkid = (set(mom_alleles) & set(kid_alleles))

In [ ]:
allele = 100

In [ ]:
wigglecommand = 10

In [ ]:
wiggle(wigglecommand)

In [ ]:
def wiggle(wigglecommand):
        if type(wigglecommand) is int:
            wiggleint = wigglecommand
            return wiggleint
        else:
            type(wigglecommand) is str
            wigglestr = int(wigglecommand)
            return wigglestr

In [ ]:
def allele_check(allele1,allele2):
    if allele1 == 'NaN' and allele2 == 'NaN':
        allele1 == 'NaN'
    elif allele1 != 'NaN' and allele2 == 'NaN':
        if allele1 >= 350:
            allele2 = 350
            allele1 = 350
        else:
            allele2 = allele1
    elif allele1 == 'NaN' and allele2 != 'NaN':
        if allele2 >= 350:
            allele1 = 350
            allele2 = 350
        else:
            allele1 = allele2
    elif allele2 >= 350:
        allele2 = 350   
    elif allele1 >= 350:
        allele1 = 350
    else:
        allele1 = allele1
        allele2 = allele2
    return allele1, allele2

In [ ]:
allele_check(50, 60)

In [ ]:
allele_range(10,150,150)

In [ ]:
def allele_range(wigglecommand, allele1, allele2):
    if allele1 and allele2 == 'NaN':
      return  
    else:
        tple1 = (allele1 - wiggle(wigglecommand), allele1 + wiggle(wigglecommand))
        tple2 = (allele2 - wiggle(wigglecommand), allele2 + wiggle(wigglecommand))
    return tple1, tple2

    

In [ ]:
def get_allele_ranges(wigglecommand, allele1, allele2):
    a, b = allele_check(allele1,allele2)
    print(a)
    print(b)
    print(allele_range(wigglecommand, a, b))
    
    

In [ ]:
get_allele_ranges(10,500,'NaN')

In [ ]:
allele_range(wigglecommand,50)

In [ ]:
def check_range(kidallele, tple):
    a,b = tple
    if a <= kidallele <= b:
        print True
    elif kidallele > a and b:
        print('Too big')
    else: 
        print('Booooo')

In [ ]:
check_range(kidallele, tple)

In [ ]:
check_range(kidallele, allele_range(wigglecommand, mom_allele1))
check_range(kidallele, allele_range(wigglecommand, mom_allele2))

In [ ]:
if check_range(kidallele, allele_range(wigglecommand, mom_allele1)) is True:
    print("Cool")
else:
    print('What is up')

In [ ]:
mom_allele2 = 200